In [1]:
# Daniel Bandala @ sep 2022
import numpy

# Levenshtein distance
Using the dynamic programming approach for calculating the Levenshtein distance, a 2-D matrix is created that holds the distances between all prefixes of the two words being compared. Given two words of lengths $m$ and $n$, and based on the steps discussed in the previous tutorial, the first thing to do is to create a 2-D matrix of integers of size $(m+1, n+1)$ or $(n+1, m+1)$ based on whether the first word represents the rows or the columns. It doesn't matter which word is used for what, but you do need to be consistent since the rest of the code depends on this choice.

In [2]:
def levenshteinDistanceMatrix(token1, token2, verbose=False):
    # init vector
    distances = numpy.zeros((len(token1) + 1, len(token2) + 1))
    for t1 in range(len(token1) + 1):
        distances[t1][0] = t1

    for t2 in range(len(token2) + 1):
        distances[0][t2] = t2
    # show distances
    if verbose: printDistances(distances, len(token1), len(token2))
    return distances

In [3]:
def printDistances(distances, token1Length, token2Length):
    for t1 in range(token1Length + 1):
        for t2 in range(token2Length + 1):
            print(int(distances[t1][t2]), end=" ")
        print()

In [4]:
levenshteinDistanceMatrix("kelm", "hello", True)

0 1 2 3 4 5 
1 0 0 0 0 0 
2 0 0 0 0 0 
3 0 0 0 0 0 
4 0 0 0 0 0 


array([[0., 1., 2., 3., 4., 5.],
       [1., 0., 0., 0., 0., 0.],
       [2., 0., 0., 0., 0., 0.],
       [3., 0., 0., 0., 0., 0.],
       [4., 0., 0., 0., 0., 0.]])

To calculate the distances between all prefixes of the two words, two for loops are used to iterate through each cell in the matrix (excluding the first row/column). Inside the loops the distances are calculated for all combinations of prefixes from the two words. Based on the discussion in the previous tutorial, the distance between two prefixes is calculated based on a 2 x 2 matrix as shown in the next figure. Such a matrix always has three known values and just one missing value which is to be calculated. If the two characters located at the end of the two prefixes being compared are equal, then the distance is equal to the value in the top-left corner of the 2 x 2 matrix. This is implemented in the next if statement. If the two characters are not equal, then the distance in the current cell is equal to the minimum of the three existing values in the 2 x 2 matrix after adding a cost of 1. An else block is added to the previous if statement to calculate such a distance according to the following code.

In [5]:
def levenshteinDistanceDP(token1, token2, verbose=False):
    distances = numpy.zeros((len(token1) + 1, len(token2) + 1))

    for t1 in range(len(token1) + 1):
        distances[t1][0] = t1

    for t2 in range(len(token2) + 1):
        distances[0][t2] = t2
        
    a = 0
    b = 0
    c = 0
    
    for t1 in range(1, len(token1) + 1):
        for t2 in range(1, len(token2) + 1):
            if (token1[t1-1] == token2[t2-1]):
                distances[t1][t2] = distances[t1 - 1][t2 - 1]
            else:
                a = distances[t1][t2 - 1]
                b = distances[t1 - 1][t2]
                c = distances[t1 - 1][t2 - 1]
                
                if (a <= b and a <= c):
                    distances[t1][t2] = a + 1
                elif (b <= a and b <= c):
                    distances[t1][t2] = b + 1
                else:
                    distances[t1][t2] = c + 1

    if verbose: printDistances(distances, len(token1), len(token2))
    return distances[len(token1)][len(token2)]

In [6]:
distance = levenshteinDistanceDP("kelm", "hello", True)

0 1 2 3 4 5 
1 1 2 3 4 5 
2 2 1 2 3 4 
3 3 2 1 2 3 
4 4 3 2 2 3 


In [7]:
print("Levenshtein distance: ",distance)

Levenshtein distance:  3.0


## Dictionary Search for Word Autocompletion and Autocorrection
One application of the Levenshtein distance is to help the writer write faster by automatically correcting typos or completing words. In this section we'll experiment with a small version of the English dictionary (which contains just 1,000 common words) to complete this task.

The dictWordDist list is sorted to leave the best-matched words at the top of the list. Inside a for loop with a number of iterations equal to the value of the numWords argument, the dictWordDist list is indexed to return a string holding the distance and the word separated by -.

In [8]:
def calcDictDistance(word, numWords):
    file = open('1-1000.txt', 'r') 
    lines = file.readlines() 
    file.close()
    dictWordDist = []
    wordIdx = 0
    
    for line in lines: 
        wordDistance = levenshteinDistanceDP(word, line.strip())        
        if wordDistance >= 10:
            wordDistance = 9
        dictWordDist.append(str(int(wordDistance)) + "-" + line.strip())
        wordIdx = wordIdx + 1

    closestWords = []
    wordDetails = []
    currWordDist = 0
    dictWordDist.sort()
    print("Best matching words: ", dictWordDist)
    for i in range(numWords):
        currWordDist = dictWordDist[i]
        wordDetails = currWordDist.split("-")
        closestWords.append(wordDetails[1])
    return closestWords

In [14]:
print(calcDictDistance("pape", 3))

Best matching words:  ['1-page', '1-paper', '2-age', '2-are', '2-base', '2-came', '2-care', '2-case', '2-ease', '2-face', '2-game', '2-gave', '2-have', '2-hope', '2-lake', '2-late', '2-made', '2-make', '2-map', '2-name', '2-pair', '2-part', '2-pass', '2-past', '2-path', '2-pay', '2-place', '2-plane', '2-pose', '2-race', '2-rope', '2-safe', '2-same', '2-save', '2-shape', '2-space', '2-take', '2-type', '2-wave', '3-a', '3-able', '3-act', '3-add', '3-ago', '3-air', '3-all', '3-am', '3-an', '3-and', '3-any', '3-apple', '3-area', '3-arm', '3-art', '3-as', '3-ask', '3-at', '3-baby', '3-back', '3-bad', '3-ball', '3-band', '3-bank', '3-bar', '3-bat', '3-be', '3-blue', '3-bone', '3-call', '3-camp', '3-can', '3-car', '3-card', '3-cat', '3-cause', '3-come', '3-copy', '3-dad', '3-dance', '3-dark', '3-day', '3-die', '3-done', '3-each', '3-ear', '3-east', '3-eat', '3-edge', '3-else', '3-eye', '3-fact', '3-fair', '3-fall', '3-far', '3-farm', '3-fast', '3-fat', '3-fine', '3-fire', '3-five', '3-free', 